# Compute quantile with val

In [1]:
import numpy as np
import torch
import math
import os
import properscoring as ps
import warnings
warnings.filterwarnings('ignore')

## SORT

In [2]:
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/val_scenes.txt"
tracking_path = "./sort/val/cfk_nlla/"

In [61]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,s,r] where x,y is the centre of the box and s is the scale/area and r is
      the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    s = w * h  # scale is just area
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, s, r])


def convert_x_to_bbox(x, score=None):
    """
    Takes a bounding box in the centre form [x,y,s,r] and returns it in the form
      [x1,y1,x2,y2] where x1,y1 is the top left and x2,y2 is the bottom right
    """
    w = np.sqrt(x[2] * x[3])
    h = x[2] / w
    if score is None:
        return np.array(
            [x[0] - w / 2.0, x[1] - h / 2.0, x[0] + w / 2.0, x[1] + h / 2.0]
        ).reshape((1, 4))
    else:
        return np.array(
            [x[0] - w / 2.0, x[1] - h / 2.0, x[0] + w / 2.0, x[1] + h / 2.0, score]
        ).reshape((1, 5))

def split_data_by_frame(data):
    res = []
    start = 0
    if data.shape[0] == 0:
        return res
    if len(data.shape) == 1:
        res.append(data[np.newaxis, :])
        return res
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas

def compute_score(data_path):
    data = np.load(data_path, allow_pickle=True)
    tp = data.item()['tp']
    scores = []
    for i in range(len(tp)):
        prediction = np.array(tp[i][0])
        pred = prediction[:,:8]
        target = np.array(tp[i][1])
        cov = prediction[:,9:]
        score = np.abs(pred-target) / np.sqrt(np.exp(cov))
        for s in score:
            scores.append(list(s))
    return np.array(scores)

def compute_quantile(scores, alpha):
    n = scores.shape[0]
    q_level = np.ceil((n+1)*(1-alpha))/n
    quantile = []
    for i in range(scores.shape[1]):
        q = np.quantile(scores[:,i], q_level, interpolation='higher')
        quantile.append(q)
    return quantile

def compute_nll(dets, gts, mode, iou_threshold):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return []
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = convert_bboxs_to_zs(matched_dets[:,:4])
    target = convert_bboxs_to_zs(matched_gts[:,:4])
    cov = matched_dets[:,8:]
    std = np.sqrt(cov)
    score = np.abs(pred-target) / std
    return score

def compute_quantile_for_mode(mode, det_path, iou_threshold, alpha):
    upper_path = det_path + mode + "/no_rsu/"
    scene_idxes_file = open(scene_idxes_file_path, "r")
    scene_idxes = [int(line.strip()) for line in scene_idxes_file]
    #print(scene_idxes)
    scores = []
    for agent in range(1,6):
        det_files_path = upper_path + "tracking" + str(agent)
        gt_files_path = gt_path + "V2X-val" + str(agent)
        for scene in scene_idxes:
            det_scene_file = os.path.join(det_files_path, str(scene) + ".txt")
            gt_scene_file = os.path.join(gt_files_path, str(scene), "gt/gt.txt")
            det_datas = np.loadtxt(det_scene_file, delimiter=",")
            gt_datas = np.loadtxt(gt_scene_file, delimiter=",")
            if len(gt_datas) == 0 or len(det_datas) == 0:
                continue
            det_datas = split_data_by_frame(det_datas)
            gt_datas = split_data_by_frame(gt_datas)
            det_idx = 0
            for gt_idx in range(len(gt_datas)):
                #print(len(det_datas[det_idx][0]), len(gt_datas[gt_idx][0]))
                # while len(np.array(det_datas[det_idx]).shape) < 2:
                #     det_idx += 1
                if det_idx >= len(det_datas):
                    #print(f"det_idx out of range, {gt_idx} {det_idx}")
                    break
                if det_datas[det_idx][0][0] != gt_datas[gt_idx][0][0]:
                    #print(f"Error idx not match {scene} {agent} {det_idx}, {gt_idx} {det_datas[det_idx][0][0]} {gt_datas[gt_idx][0][0]}")
                    continue
                score_list = compute_nll(det_datas[det_idx], gt_datas[gt_idx], mode, iou_threshold)
                scores.extend(score_list)
                det_idx += 1
    scores = np.array(scores)
    quantile = compute_quantile(scores, alpha)
    return quantile

def show_one_result(path, iou_threshold):
    confident_range = 0.6827
    alpha = 1-confident_range
    mode_list = ["disco", "upperbound","lowerbound"]
    for mode in mode_list:
        quantile = compute_quantile_for_mode(mode, path, iou_threshold, alpha)
        print(f"quantile for {mode} is {quantile}")
        print_var_quantile_1(quantile)

def print_var_quantile(q):
    sq = np.array(q) /3
    print(list(sq*sq))

def print_var_quantile_1(q):
    sq = np.array(q)
    print(list(sq*sq))

def print_var_quantile_2(q):
    sq = np.array(q)/2
    print(list(sq*sq))
    
def show_one_result_2(path, iou_threshold):
    confident_range = 0.9545
    alpha = 1-confident_range
    mode_list = ["disco", "upperbound","lowerbound"]
    for mode in mode_list:
        quantile = compute_quantile_for_mode(mode, path, iou_threshold, alpha)
        print(f"quantile for {mode} is {quantile}")
        print_var_quantile_2(quantile)

In [32]:
confident_range = 0.9973
compute_quantile_for_mode("upperbound", tracking_path, 0.5, 1-confident_range)

[9.26636665530604, 26.389540552570978, 980.3717595051038, 47.46338715908096]

In [48]:
print_var_quantile([9.26636665530604, 26.389540552570978, 980.3717595051038, 47.46338715908096])

[9.54061677672974, 77.37865006397645, 106792.08742612592, 250.3081245125346]


In [34]:
confident_range = 0.9973
compute_quantile_for_mode("disco", tracking_path, 0.5, 1-confident_range)

[7.257887314355021, 25.74134829288509, 1095.7692268573962, 36.05259130092381]

In [49]:
print_var_quantile([7.257887314355021, 25.74134829288509, 1095.7692268573962, 36.05259130092381])

[5.852992029763949, 73.6241124372909, 133412.24428085063, 144.42103772349412]


In [36]:
confident_range = 0.9973
compute_quantile_for_mode("lowerbound", tracking_path, 0.5, 1-confident_range)

[8.711900100131754, 23.213263898179875, 1068.081898446353, 10.755487829590209]

In [50]:
print_var_quantile([8.711900100131754, 23.213263898179875, 1068.081898446353, 10.755487829590209])

[8.433022594963962, 59.87284675628234, 126755.43797652953, 12.853390939162566]


In [58]:
show_one_result(tracking_path, 0.5)

quantile for disco is [1.2587643535709336, 4.105097014062292, 1095.1754273107442, 0.09232735611344699]
[1.5844876978208504, 16.85182149486315, 1199409.2165852713, 0.008524340686899256]
quantile for upperbound is [1.4201529360783647, 4.54480541660601, 1048.5368111993635, 0.09341083560376137]
[2.0168343618519997, 20.655256274811332, 1099429.4444401297, 0.008725584208192933]
quantile for lowerbound is [0.9745005771422002, 3.795479075719574, 474.2240417007059, 0.08644231115879157]
[0.9496513748504812, 14.405661414225111, 224888.44172695288, 0.007472273158473342]


In [62]:
show_one_result_2(tracking_path, 0.5)

quantile for disco is [4.531937515344233, 12.436723035701078, 3669.607672931809, 1.169126199270564]
[5.134614410746115, 38.66801996668446, 3366505.118310001, 0.3417140174552086]
quantile for upperbound is [4.767704087374154, 12.525844966472732, 3381.40135689267, 1.1668234057518179]
[5.6827505661910545, 39.22419803102757, 2858468.784098897, 0.34036921505256784]
quantile for lowerbound is [4.2251913608151535, 11.089558010883625, 1740.040663279948, 0.6954586940303473]
[4.463060508876752, 30.744574219188294, 756935.3774669303, 0.12091569877559906]


## ByteTrack

In [40]:
gt_path = "./TrackEval/data/gt/mot_challenge/"
scene_idxes_file_path = "../utils/val_scenes.txt"
tracking_path = "./byte_tracker/val/cfk_nlla/"

In [63]:
def convert_bbox_to_z(bbox):
    """
    Takes a bounding box in the form [x1,y1,x2,y2] and returns z in the form
      [x,y,r,h] where x,y is the centre of the box and r is
      the aspect ratio
    """
    w = bbox[2] - bbox[0]
    h = bbox[3] - bbox[1]
    x = bbox[0] + w / 2.0
    y = bbox[1] + h / 2.0
    if float(h) < 0.00001:
        print([x, y, w, h])
        print(bbox)
    r = w / float(h)
    return np.array([x, y, r, h])

def split_data_by_frame(data):
    res = []
    start = 0
    if data.shape[0] == 0:
        return res
    if len(data.shape) == 1:
        res.append(data[np.newaxis, :])
        return res
    for i in range(1,data.shape[0]):
        if data[i][0] != data[start][0]:
            res.append(data[start:i,:])
            start = i
    res.append(data[start:,:])
    return res

def iou_batch(bb_test, bb_gt):
    """
    From SORT: Computes IOU between two bboxes in the form [x1,y1,x2,y2]
    """
    bb_gt = np.expand_dims(bb_gt, 0)
    bb_test = np.expand_dims(bb_test, 1)

    xx1 = np.maximum(bb_test[..., 0], bb_gt[..., 0])
    yy1 = np.maximum(bb_test[..., 1], bb_gt[..., 1])
    xx2 = np.minimum(bb_test[..., 2], bb_gt[..., 2])
    yy2 = np.minimum(bb_test[..., 3], bb_gt[..., 3])
    w = np.maximum(0.0, xx2 - xx1)
    h = np.maximum(0.0, yy2 - yy1)
    wh = w * h
    o = wh / (
        (bb_test[..., 2] - bb_test[..., 0]) * (bb_test[..., 3] - bb_test[..., 1])
        + (bb_gt[..., 2] - bb_gt[..., 0]) * (bb_gt[..., 3] - bb_gt[..., 1])
        - wh
    )
    return o

def linear_assignment(cost_matrix):
    try:
        import lap

        _, x, y = lap.lapjv(cost_matrix, extend_cost=True)
        return np.array([[y[i], i] for i in x if i >= 0])  #
    except ImportError:
        from scipy.optimize import linear_sum_assignment

        x, y = linear_sum_assignment(cost_matrix)
        return np.array(list(zip(x, y)))

def pre_process_bbox(datas):
    """
    make the bbox be [x1,y1,x2,y2] where [x1,y1] is the left bottom and [x2,y2] is the right top
    """
    for i in range(datas.shape[0]):
        small = [min(datas[i][0], datas[i][2]), min(datas[i][1], datas[i][3])]
        large = [max(datas[i][0], datas[i][2]), max(datas[i][1], datas[i][3])]
        datas[i][0] = small[0]
        datas[i][1] = small[1]
        datas[i][2] = large[0]
        datas[i][3] = large[1]
    return datas

def remove_error_bbox(dets):
    res = []
    for i in range(dets.shape[0]):
        if dets[i, 0] == dets[i, 2] or dets[i, 1] == dets[i, 3]:
            continue
        res.append(list(dets[i]))
    return np.array(res)

def convert_bboxs_to_zs(datas):
    for i in range(datas.shape[0]):
        datas[i, :4] = convert_bbox_to_z(datas[i,:4])
    return datas
    
def compute_nll(dets, gts, mode, iou_threshold):
    dets = pre_process_bbox(dets[:,2:])
    gts = pre_process_bbox(gts[:,2:])
    dets = remove_error_bbox(dets)
    iou_matrix = iou_batch(dets, gts)
    if min(iou_matrix.shape) > 0:
        a = (iou_matrix > iou_threshold).astype(np.int32)
        if a.sum(1).max() == 1 and a.sum(0).max() == 1:
            matched_indices = np.stack(np.where(a), axis=1)
        else:
            matched_indices = linear_assignment(-iou_matrix)
    else:
        matched_indices = np.empty(shape=(0, 2))
    if len(matched_indices) == 0:
        print("Error length of matched_indices if zero!")
    if iou_threshold >= 0.5:
        matches = []
        for m in matched_indices:
            if iou_matrix[m[0], m[1]] >= iou_threshold:
                matches.append([m[0], m[1]])
        matched_indices = np.array(matches)
    if len(np.array(matched_indices).shape) < 2:
        return []
    matched_dets = dets[matched_indices[:,0]]
    matched_gts = gts[matched_indices[:,1]]
    pred = convert_bboxs_to_zs(matched_dets[:,:4])
    target = convert_bboxs_to_zs(matched_gts[:,:4])
    cov = matched_dets[:,8:]
    std = np.sqrt(cov)
    score = np.abs(pred-target) / std
    return score

In [41]:
confident_range = 0.9973
compute_quantile_for_mode("upperbound", tracking_path, 0.5, 1-confident_range)

[14.675761481491042, 29.097820609051176, 66.64290152805684, 30.913460464033168]

In [51]:
print_var_quantile([14.675761481491042, 29.097820609051176, 66.64290152805684, 30.913460464033168])

[23.930886117957346, 94.07590713294702, 493.47514711980904, 106.18244865126019]


In [43]:
confident_range = 0.9973
compute_quantile_for_mode("disco", tracking_path, 0.5, 1-confident_range)

[14.334850052114051, 33.86344301502989, 44.95038087908978, 30.87056006683162]

In [52]:
print_var_quantile([14.334850052114051, 33.86344301502989, 44.95038087908978, 30.87056006683162])

[22.831991779621582, 127.4147525369085, 224.50408235280446, 105.88794209331766]


In [45]:
confident_range = 0.9973
compute_quantile_for_mode("lowerbound", tracking_path, 0.5, 1-confident_range)

[14.236715334025112, 26.094051875076637, 14.29643821802983, 33.207487831573594]

In [53]:
print_var_quantile([14.236715334025112, 26.094051875076637, 14.29643821802983, 33.207487831573594])

[22.520451500229527, 75.65550480657672, 22.709793969104886, 122.52636089823424]


In [60]:
show_one_result(tracking_path, 0.5)

quantile for disco is [1.2587643535709336, 4.105097014062292, 0.08813197452271423, 3.27674215084452]
[1.5844876978208504, 16.85182149486315, 0.007767244933272351, 10.737039123121171]
quantile for upperbound is [1.4201529360783647, 4.54480541660601, 0.08100396417048444, 3.1321268748693316]
[2.0168343618519997, 20.655256274811332, 0.0065616422113331276, 9.810218760278726]
quantile for lowerbound is [0.9745005771422002, 3.795479075719574, 0.039849019116122125, 2.561358615555389]
[0.9496513748504812, 14.405661414225111, 0.0015879443245170666, 6.56055795747982]


In [64]:
show_one_result_2(tracking_path, 0.5)

quantile for disco is [4.531937515344233, 12.436723035701078, 1.101188355046245, 9.5705881987947]
[5.134614410746115, 38.66801996668446, 0.3031539483223637, 22.899039617727098]
quantile for upperbound is [4.767704087374154, 12.525844966472732, 1.0581315990618603, 9.762428737517125]
[5.6827505661910545, 39.22419803102757, 0.2799106202333024, 23.82625371377505]
quantile for lowerbound is [4.2251913608151535, 11.089558010883625, 0.33743600506791954, 8.700059608265224]
[4.463060508876752, 30.744574219188294, 0.028465764379049255, 18.922759296842013]
